## Imports

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

# sequenceTest.py

Contains code from relevant file

In [2]:
# Splitting data into banches so we can actually process this thing.
class DataSequence(keras.utils.Sequence):

    def __init__(self, file, batch_size):
        self.file = file
        self.batch_size = batch_size
      
    def __len__(self):
        #total_length = sum(1 for row in open(self.file))
        total_length = 100_000
        return int(np.ceil(total_length) / self.batch_size)
    
    
    def __getitem__(self, idx):
        df = pd.read_csv('../data/createdData/preprocessedDataset.csv', nrows = self.batch_size, skiprows=idx*self.batch_size, dtype=np.int8)

        X = df.iloc[:, :-1]  # Select all columns except the last one (the features)
        y = df.iloc[:, -1]   # Select the last column (the target)

        X = X.to_numpy()
        y = y.to_numpy()
        
        return (X, y)

# training.py

Contains code from relevant file

In [3]:
# Define the neural network architecture
model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(13, 8, 8)),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    tf.keras.layers.Dense(832, activation='linear'),
    tf.keras.layers.Dense(832, activation='relu'),
    #tf.keras.layers.Dense(1, activation=scaled_sigmoid)
    tf.keras.layers.Dense(1, activation='linear')
])
# Compile the model
model.compile(optimizer='adam',
              loss='MeanAbsoluteError')

### Trying to used 'vectorized.csv'

It is not returning in exactly the way we'd like, so I'm trying to deal with this here. Idk if this will work, that's why I need to try and plug it in to the neural network

    UPDATE 12:03 PM 4/28:
    
    Doesn't work, but figured out why and important information about the data. FEN string arrays are stored such that every single bit has its own column. That is, there are 832 columns we pass into the neural network. Makes sense in hindsight, but it was not something we had nailed down up to this point. This should make understanding the "failed to convert to tensor from numpyarray" or whatever a lot easier to understand

Back to fitting the model

In [4]:
# Using DataSequence
# train_sequence = DataSequence('../data/createdData/preprocessedDataset.csv', 10_000)

# # Train the model
# model.fit(train_sequence, epochs=10)

In [5]:
df = pd.read_csv('../data/createdData/preprocessedDataset.csv', nrows = 4_000_000, dtype=np.int8)

X = df.iloc[:, :-1]  # Select all columns except the last one (the features)
y = df.iloc[:, -1]   # Select the last column (the target)

X = X.to_numpy()
y = y.to_numpy()

model.fit(X, y, epochs=25, batch_size=1_000)

model.save('../saved_models/saved_model_4mRows_25Epochs_1kBatchSize')

Epoch 1/25
4000/4000 [==============================] - 35s 8ms/step - loss: 7.5533
Epoch 2/25
4000/4000 [==============================] - 32s 8ms/step - loss: 6.1777
Epoch 3/25
4000/4000 [==============================] - 32s 8ms/step - loss: 5.7457
Epoch 4/25
4000/4000 [==============================] - 32s 8ms/step - loss: 5.4939
Epoch 5/25
4000/4000 [==============================] - 32s 8ms/step - loss: 5.3181
Epoch 6/25
4000/4000 [==============================] - 32s 8ms/step - loss: 5.1948
Epoch 7/25
4000/4000 [==============================] - 32s 8ms/step - loss: 5.0717
Epoch 8/25
4000/4000 [==============================] - 31s 8ms/step - loss: 4.9716
Epoch 9/25
4000/4000 [==============================] - 32s 8ms/step - loss: 4.9054
Epoch 10/25
4000/4000 [==============================] - 32s 8ms/step - loss: 4.8421
Epoch 11/25
4000/4000 [==============================] - 32s 8ms/step - loss: 4.7871
Epoch 12/25
4000/4000 [==============================] - 34s 8ms/step - lo